# Power outages with state & time info
> Author: [Dawn Graham](https://dawngraham.github.io/)

The first part of this notebook assigns location of power outages (as state abbreviations) to power outages. It then transforms event and restoration info to datetime values.

The last adds a column to tweets data indicating if the tweet was at the same time as a verified outage.

Versions used:
- Python 3.6.6
- pandas 0.23.4
- numpy 1.15.4

### Import libraries

In [1]:
import pandas as pd
import numpy as np
from collections import OrderedDict

### Get data

In [2]:
df = pd.read_csv('../data/Grid_Disruption_00_14_standardized_cleaned.csv')
df.head()

,Event Description,Year,Date Event Began,Time Event Began,Date of Restoration,Time of Restoration,Respondent,Geographic Areas,NERC Region,Demand Loss (MW),Number of Customers Affected,Tags
0,Severe Weather - Thunderstorms,2014,6/30/2014,8:00 PM,7/2/2014,6:30 PM,Exelon Corporation/ComEd,Illinois,RFC,Unknown,420000.0,"severe weather, thunderstorm"
1,Severe Weather - Thunderstorms,2014,6/30/2014,11:20 PM,7/1/2014,5:00 PM,Northern Indiana Public Service Company,North Central Indiana,RFC,Unknown,127000.0,"severe weather, thunderstorm"
2,Severe Weather - Thunderstorms,2014,6/30/2014,5:55 PM,7/1/2014,2:53 AM,We Energies,Southeast Wisconsin,MRO,424,120000.0,"severe weather, thunderstorm"
3,Fuel Supply Emergency - Coal,2014,6/27/2014,1:21 PM,Unknown,Unknown,We Energies,Wisconsin,MRO,Unknown,NaN,"fuel supply emergency, coal"
4,Physical Attack - Vandalism,2014,6/24/2014,2:54 PM,6/24/2014,2:55 PM,Tennessee Valley Authority,"Nashville, Tennessee",SERC,Unknown,NaN,"vandalism, physical"


In [3]:
df.shape

(1639, 12)

### Drop rows
Drop rows with missing geographic data.

In [4]:
# # Look at unique Geographic Areas
# sorted(df['Geographic Areas'].astype(str).unique())

In [5]:
df = df[df['Geographic Areas'] != '-']
df = df[df['Geographic Areas'].notnull()]

### State abbreviations dictionary
Bring in dictionary of state names and abbreviations. From [mGalarnyk/us_state_abbrev.py](https://gist.github.com/mGalarnyk/27c99c5f55133a2ceb42b8c9a450d794).

In [6]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    'Puerto Rico': 'PR',
}

### Create `State` column

In [7]:
# Set up empty column
df['State'] = ''

If `Geographic Areas` contains state name or abbreviation, include abbreviation in `State` column.

In [8]:
for state in us_state_abbrev:
    df.loc[df['Geographic Areas'].str.contains(state), 'State'] += f'{us_state_abbrev[state]} '
    df.loc[((df['Geographic Areas'].str.contains(us_state_abbrev[state])) & (us_state_abbrev[state] not in df['State'])), 'State'] += f'{us_state_abbrev[state]} '

Fill in missing values in `State` column.

In [9]:
df.loc[(df['Geographic Areas'].str.contains('Bakersfield') & ('CA ' not in df['State'])), 'State'] += f'CA '
df.loc[(df['Geographic Areas'].str.contains('Los Angeles') & ('CA ' not in df['State'])), 'State'] += f'CA '
df.loc[(df['Geographic Areas'].str.contains('Sacramento') & ('CA ' not in df['State'])), 'State'] += f'CA '
df.loc[(df['Geographic Areas'].str.contains('San Diego') & ('CA ' not in df['State'])), 'State'] += f'CA '
df.loc[(df['Geographic Areas'].str.contains('San Francisco') & ('CA ' not in df['State'])), 'State'] += f'CA '
df.loc[(df['Geographic Areas'].str.contains('Connecicut') & ('CT ' not in df['State'])), 'State'] += f'CT '
df.loc[(df['Geographic Areas'].str.contains('Maui') & ('HI ' not in df['State'])), 'State'] += f'HI '
df.loc[(df['Geographic Areas'].str.contains('Oahu') & ('HI ' not in df['State'])), 'State'] += f'HI '
df.loc[(df['Geographic Areas'].str.contains('Akron') & ('OH ' not in df['State'])), 'State'] += f'OH '
df.loc[(df['Geographic Areas'].str.contains('Cleveland') & ('OH ' not in df['State'])), 'State'] += f'OH '
df.loc[(df['Geographic Areas'].str.contains('Long Island') & ('NY ' not in df['State'])), 'State'] += f'NY '
df.loc[(df['Geographic Areas'].str.contains('Houston') & ('TX ' not in df['State'])), 'State'] += f'TX '

df.loc[(df['Respondent'].str.contains('California') & ('CA ' not in df['State'])), 'State'] += f'CA '
df.loc[(df['Respondent'].str.contains('Florida') & ('FL ' not in df['State'])), 'State'] += f'FL '
df.loc[(df['Respondent'].str.contains('Detroit') & ('MI ' not in df['State'])), 'State'] += f'MI '
df.loc[(df['Respondent'].str.contains('Texas') & ('TX ' not in df['State'])), 'State'] += f'TX '
df.loc[(df['Respondent'].str.contains('Puget Sound') & ('WA ' not in df['State'])), 'State'] += f'WA '
df.loc[(df['Respondent'].str.contains('Puerto Rico') & ('PR ' not in df['State'])), 'State'] += f'PR '

Correct incorrect or missing `State` values.

In [10]:
df.loc[df['Geographic Areas'].str.contains('VA')]
df.loc[1009, 'State'] = 'TN '

In [11]:
df.loc[876, 'State'] = 'CO '
df.loc[1006, 'State'] = 'KY '
df.loc[1073, 'State'] = 'WA '
df.loc[1107, 'State'] = 'CT ME MA NH RI VT '
df.loc[1139, 'State'] = 'FL '
df.loc[1210, 'State'] = 'AZ '

Update rows that included were flagged as `VA` because of West Virginia.

In [12]:
df.loc[12, 'State'] = 'WV '
df.loc[76, 'State'] = 'MD WV '
df.loc[317, 'State'] = 'WV '
df.loc[328, 'State'] = 'MD WV '
df.loc[402, 'State'] = 'MD WV '
df.loc[405, 'State'] = 'IN MI OH WV '
df.loc[410, 'State'] = 'WV '
df.loc[693, 'State'] = 'KY WV '
df.loc[733, 'State'] = 'KY WV '
df.loc[836, 'State'] = 'OH WV KY '
df.loc[836, 'State'] = 'PA WV '
df.loc[918, 'State'] = 'WV KY OH '
df.loc[991, 'State'] = 'KY WV OH '
df.loc[1340, 'State'] = 'MD WV PA '

## Check remaining rows with no `State` value.

In [13]:
df[df['State'] == ''].shape

(44, 13)

In [14]:
df[df['State'] == '']

,Event Description,Year,Date Event Began,Time Event Began,Date of Restoration,Time of Restoration,Respondent,Geographic Areas,NERC Region,Demand Loss (MW),Number of Customers Affected,Tags,State
82,Physical Attack - Vandalism,2014,1/29/2014,4:00 PM,1/29/2014,4:44 PM,Madison Gas and Electric,Unknown,MRO,Unknown,NaN,"vandalism, physical",
99,Electrical System Islanding,2014,1/18/2014,5:39 PM,Unknown,Unknown,First Energy Solutions Corp.,Unknown,RFC,Unknown,NaN,islanding,
106,Voltage Reduction due to Severe Weather - Cold,2014,1/8/2014,5:00 AM,1/8/2014,6:30 AM,American Electric Power,Unknown,RFC,576,NaN,"severe weather, cold",
112,Public Appeal due to Severe Weather - Cold,2014,1/7/2014,9:00 PM,1/8/2014,9:00 AM,PJM Interconnection,Unknown,RFC,Unknown,NaN,"severe weather, cold",
117,Public Appeal due to Severe Weather - Cold,2014,1/6/2014,8:45 PM,1/7/2014,9:00 PM,PJM Interconnection,Unknown,RFC,Unknown,NaN,"severe weather, cold",
120,Voltage Reduction due to Severe Weather - Cold,2014,1/6/2014,7:50 PM,1/6/2014,8:44 PM,Potomac Electric Power Co,District of Columbia,RFC,Unknown,NaN,"severe weather, cold",
121,Voltage Reduction due to Severe Weather - Cold,2014,1/6/2014,7:50 PM,1/6/2014,8:44 PM,PJM Interconnection,Unknown,RFC,Unknown,NaN,"severe weather, cold",
161,Electrical System Separation (Islanding),2013,10/21/2013,5:18 AM,10/21/2013,5:33 AM,Pacific Gas & Electric Co,Location Unknown,WECC,115,433.0,islanding,
185,Electrical System Separation (Islanding); Seve...,2013,8/5/2013,6:35 PM,8/5/2013,6:45 PM,WECC RC Vancouver,"Vancouver, British Columbia",WECC,Unknown,NaN,"islanding, severe weather",
187,Electrical System Separation (Islanding,2013,8/1/2013,6:54 PM,8/1/2013,7:37 PM,WECC RC Vancouver,Western British Columbia,WECC,420,NaN,islanding,


## Get begin / end times

In [15]:
df['Year'].value_counts()

2011    307
2012    193
2013    170
2008    148
2014    125
2010    123
2009     97
2004     93
2006     91
2005     85
2007     78
2003     60
2000     29
2002     23
2001     15
Name: Year, dtype: int64

In [16]:
# Quick filter to only use data from 2009 - 2012
df = df[(df['Year'] >= 2009) & (df['Year'] <= 2012)]

In [17]:
# Check to see if any begin/end times are missing
df.isnull().sum()

Event Description                 0
Year                              0
Date Event Began                  0
Time Event Began                  0
Date of Restoration               0
Time of Restoration               0
Respondent                        0
Geographic Areas                  0
NERC Region                       2
Demand Loss (MW)                232
Number of Customers Affected    168
Tags                              0
State                             0
dtype: int64

### Get datetime values

In [18]:
# Combine Began date & time into 'Outage Start' as datetime
df.loc[:,'Outage Start'] = pd.to_datetime(df['Date Event Began'].astype(str)+' '+df['Time Event Began'].astype(str))

In [19]:
# Drop the two rows where 'Date of Restoration' == 'Ongoing'
df = df[df['Date of Restoration'] != 'Ongoing']

In [20]:
# Combine Restoration date & time into 'Outage End' as datetime
df.loc[:,'Outage End'] = pd.to_datetime(df['Date of Restoration'].astype(str)+' '+df['Time of Restoration'].astype(str))

In [21]:
# Save only desired columns to outages.csv
df = df[['Respondent', 'Geographic Areas', 'State', 'Number of Customers Affected', 'Outage Start', 'Outage End']]
df.to_csv('./data/outages.csv')
df.head(3)

,Respondent,Geographic Areas,State,Number of Customers Affected,Outage Start,Outage End
295,City of Washington - (NC),North Carolina,NC,12000.0,2012-12-31 14:21:00,2012-12-31 16:30:00
296,Town of Stantonsburg - (NC),"Stantonsburg, North Carolina",NC,1200.0,2012-12-26 14:50:00,2012-12-26 19:40:00
297,CenterPoint Energy,"Houston, Texas",TX TX,262000.0,2012-12-25 09:28:00,2012-12-26 16:28:00


---
## Check if tweets coincide with power outages

In [22]:
tweets = pd.read_csv('../data/combined_monthlytweets.csv')

# Set `timestamp` to datetime and set it to index
tweets['timestamp'] = pd.to_datetime(tweets['timestamp'])
tweets.set_index('timestamp', inplace=True)
tweets.head(3)

,id,likes,query,replies,retweets,text,user
timestamp,,,,,,,
2012-11-01 23:50:22,264152432282578945,1,EversourceMA OR EversourceNH OR VelcoVT OR nat...,1.0,3,"Tom May, CEO of Northeast Utilities, the paren...",EversourceMA
2012-11-01 23:45:13,264151136792109056,0,EversourceMA OR EversourceNH OR VelcoVT OR nat...,0.0,0,@NYGovCuomo @lipanews @nationalgridus @nyseand...,readyforthenet
2012-11-01 23:34:44,264148498352590849,1,EversourceMA OR EversourceNH OR VelcoVT OR nat...,0.0,1,Some amazing video from the Wareham microburst...,EversourceMA


In [23]:
# Dataframe contains sponsored tweets outside of our time of interest (September 2009 to end of December 2012).
# Filtering to dates of interest.
tweets = tweets[:'2012']

In [24]:
# Set up 'outage' column with all values set to 0 and empty 'outage_state' column
tweets['outage'] = 0
tweets['outage_state'] = ''

In [25]:
tweets.shape

(38069, 9)

In [29]:
count = 0
for i in range(len(tweets)):
    count += 1
    for row in range(len(df)):
        if ((df.iloc[row]['Outage Start'] <= tweets.index[i]) & (tweets.index[i] <= df.iloc[row]['Outage End'])):
#             ## Print to confirm tweet times are during an outage
#             print('Outage start:', df.iloc[row]['Outage Start'])
#             print('Tweet time:  ', tweets.index[i])
#             print('Outage end:  ', df.iloc[row]['Outage End'])
#             print('Outage state: ', df.iloc[row]['State'])
#             print('')
            tweets.loc[tweets.index[i], 'outage'] = 1
            tweets.loc[tweets.index[i], 'outage_state'] = tweets['outage_state'][i] + ' ' + f"{df.iloc[row]['State']}"
#             break
    if (count) % 100 == 0:
         print(count, end=' ')

100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500 5600 5700 5800 5900 6000 6100 6200 6300 6400 6500 6600 6700 6800 6900 7000 7100 7200 7300 7400 7500 7600 7700 7800 7900 8000 8100 8200 8300 8400 8500 8600 8700 8800 8900 9000 9100 9200 9300 9400 9500 9600 9700 9800 9900 10000 10100 10200 10300 10400 10500 10600 10700 10800 10900 11000 11100 11200 11300 11400 11500 11600 11700 11800 11900 12000 12100 12200 12300 12400 12500 12600 12700 12800 12900 13000 13100 13200 13300 13400 13500 13600 13700 13800 13900 14000 14100 14200 14300 14400 14500 14600 14700 14800 14900 15000 15100 15200 15300 15400 15500 15600 15700 15800 15900 16000 16100 16200 16300 16400 16500 16600 16700 16800 16900 17000 17100 17200 17300 17400 17500 17600 17700 17800 17900 18000 18100 18200 18300 18400 1850

In [33]:
# Remove duplicates in `outage state` column
tweets['outage_state'] = tweets['outage_state'].str.split()\
         .apply(lambda x: OrderedDict.fromkeys(x).keys()).str.join(' ')

In [37]:
tweets.head()

,id,likes,query,replies,retweets,text,user,outage,outage_state
timestamp,,,,,,,,,
2012-11-01 23:50:22,264152432282578945,1,EversourceMA OR EversourceNH OR VelcoVT OR nat...,1.0,3,"Tom May, CEO of Northeast Utilities, the paren...",EversourceMA,1,WV OH PA NJ CT MA NY DE MD IN KY MI
2012-11-01 23:45:13,264151136792109056,0,EversourceMA OR EversourceNH OR VelcoVT OR nat...,0.0,0,@NYGovCuomo @lipanews @nationalgridus @nyseand...,readyforthenet,1,WV OH PA NJ CT MA NY DE MD IN KY MI
2012-11-01 23:34:44,264148498352590849,1,EversourceMA OR EversourceNH OR VelcoVT OR nat...,0.0,1,Some amazing video from the Wareham microburst...,EversourceMA,1,WV OH PA NJ CT MA NY DE MD IN KY MI
2012-11-01 23:34:20,264148399190851584,0,EversourceMA OR EversourceNH OR VelcoVT OR nat...,0.0,0,@nationalgridus Call me if you need some help ...,sparky1000,1,WV OH PA NJ CT MA NY DE MD IN KY MI
2012-11-01 23:31:56,264147793147490304,0,EversourceMA OR EversourceNH OR VelcoVT OR nat...,1.0,8,Current PSNH statewide w/o power: 885. We're d...,EversourceNH,1,WV OH PA NJ CT MA NY DE MD IN KY MI


In [35]:
tweets['outage'].value_counts()

1    20431
0    17638
Name: outage, dtype: int64

In [36]:
tweets.to_csv('../data/combined_tweets_outages.csv')